In [ ]:
# API invocation test
import os
from openai import OpenAI   # openai==1.7.0
import base64
import io
from PIL import Image

# from dotenv import load_dotenv
# load_dotenv()

# Set OpenAI API key (can also be loaded from environment variables)
api_key = 'sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'  # Your API key


def encode_image(image_path, target_size):
    """Resize an image and encode it into Base64 format"""
    with Image.open(image_path) as img:
        # Convert RGBA images to RGB
        if img.mode == "RGBA":
            img = img.convert("RGB")
        img_resized = img.resize(target_size)

    with io.BytesIO() as buffer:
        img_resized.save(buffer, format="JPEG")
        return base64.b64encode(buffer.getvalue()).decode('utf-8')


def encode_images_in_folder(folder_path, target_size=(256, 256)):
    """Encode all images in the specified folder into Base64 format"""
    encoded_images = []
    files = os.listdir(folder_path)
    image_files = [
        f for f in files
        if f.endswith('.jpeg') or f.endswith('.jpg') or f.endswith('.png')
    ]

    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        encoded_images.append(encode_image(image_path, target_size))

    return encoded_images


# Path to the image folder
folder_path = r"C:\Users\Ziv\Python_Projects\Project8_Autodefect\autodefect\auto_annotation\dataset\small_test"

# Encode all images in the folder
encoded_images = encode_images_in_folder(folder_path)  # encoded_images[i]

# for i in range(len(encoded_images)):
#     # print(f"Encoded {i}: {encoded_images[i]}...")  # Print full Base64 string
#     print(f"Encoded {i}: ...")  # Print placeholder only


# Initialize OpenAI API client
client = OpenAI(api_key=api_key)
# client = OpenAI(api_key=api_key, base_url='https://www.xxxxxxx')  # Use base_url if using an API proxy


# Construct the multi-modal input content
content = [
    {
        "type": "text",
        "text": "What are in these images? Is there any difference between them?"
    }
]

# Append each encoded image to the content
for image in encoded_images:
    content.append({
        "type": "image_url",
        "image_url": {"url": f"data:image/jpeg;base64,{image}"}
    })


# Send the request to the model
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": content,
        }
    ],
    model="gpt-4o-mini",
    max_tokens=200,
)

# Print the model response
print(response.choices[0].message)


In [ ]:
# FBP feature-augmented prompts for ZSR
import os
import re
import shutil
import base64
import io
import csv
# from dotenv import load_dotenv
from PIL import Image
from openai import OpenAI

##### Automatic retry when API token expires #####
import time
import random


def retry_with_backoff(func, max_retries=5, base_delay=3, backoff_factor=2):
    """Retry execution with exponential backoff when an exception occurs"""
    def wrapper(*args, **kwargs):
        delay = base_delay
        for attempt in range(max_retries):
            try:
                return func(*args, **kwargs)
            except Exception as e:
                print(f"⚠️ Attempt {attempt + 1} failed: {e}")
                if attempt < max_retries - 1:
                    sleep_time = delay + random.uniform(0, 1)
                    print(f"⏳ Retrying after {sleep_time:.1f}s...")
                    time.sleep(sleep_time)
                    delay *= backoff_factor
                else:
                    print("❌ Maximum retry attempts reached. Skipping.")
                    return None
    return wrapper


# # Load environment variables
# load_dotenv()
api_key = 'sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'  # Your API key

if not api_key:
    raise ValueError("API key is not configured. Please set OPENAI_API_KEY in the .env file.")

client = OpenAI(api_key=api_key)
# client = OpenAI(api_key=api_key, base_url='https://www.xxxxxxx')  # Use base_url if using an API proxy


# ==================== Visual feature prompts for defect categories (FBP core) ====================
DEFECT_FEATURE_HINT = (
    "Crack: appears as a line or gap in concrete or material.\n"
    "Efflorescence: whitish powdery or crystalline deposits caused by salts.\n"
    "Corrosion: rusty, eroded, or oxidized surfaces, especially on metal parts.\n"
    "Scaling: surface flaking, peeling, or exfoliation on concrete or plaster."
)


# ==================== Image encoding ====================
def encode_image(image_path, target_size):
    """Resize an image and encode it into Base64 format"""
    with Image.open(image_path) as img:
        if img.mode == "RGBA":
            img = img.convert("RGB")
        img_resized = img.resize(target_size)
    with io.BytesIO() as buffer:
        img_resized.save(buffer, format="JPEG")
        return base64.b64encode(buffer.getvalue()).decode('utf-8')


def encode_images_in_folder(folder_path, target_size=(256, 256)):
    """Encode all images in a folder into Base64 format"""
    encoded_images = []
    files = os.listdir(folder_path)
    image_files = [f for f in files if f.endswith(('.jpeg', '.jpg', '.png', '.JPG'))]
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        encoded_images.append(encode_image(image_path, target_size))
    return encoded_images, image_files


# ==================== Image description (with feature prompts) ====================
@retry_with_backoff
def generate_description(content):
    """Generate an image description using the LLM"""
    return client.chat.completions.create(
        messages=[{"role": "user", "content": content}],
        model="gpt-4o-mini",
        max_tokens=200,
    )


def describe_image_separately(encoded_images, image_files):
    """Generate descriptions for images individually"""
    descriptions = {}

    for i, image in enumerate(encoded_images):
        image_file = image_files[i]
        content = [
            {
                "type": "text",
                "text":
                    "Please describe the main defect in this image, considering the following visual features "
                    "of building surface defects:\n"
                    + DEFECT_FEATURE_HINT + "\n\n"
                    + "Describe the observed defect based on the visible characteristics."
            },
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image}"}
            }
        ]

        response = generate_description(content)

        if response and hasattr(response, "choices"):
            description = response.choices[0].message.content.strip()
            print(f"Description response (file: {image_file}): {description}")
            descriptions[image_file] = description
        else:
            print(f"Failed to generate description (file: {image_file})")
            descriptions[image_file] = "Description generation failed"

    return descriptions


def describe_images_in_batches(encoded_images, image_files, batch_size=5):
    """Generate image descriptions in batches"""
    descriptions = {}
    num_images = len(encoded_images)

    for i in range(0, num_images, batch_size):
        batch_images = encoded_images[i:i + batch_size]
        batch_files = image_files[i:i + batch_size]
        batch_descriptions = describe_image_separately(batch_images, batch_files)
        descriptions.update(batch_descriptions)

    return descriptions


# ==================== Classification (with feature prompts) ====================
def classify_and_describe_image(description):
    """Classify an image based on its description and provide supporting evidence"""
    prompt = (
        "The following is a description of a surface defect image. "
        "Please classify it into one of the following categories based on the described visual features:\n\n"
        f"{DEFECT_FEATURE_HINT}\n\n"
        f"Description: {description}\n\n"
        "Select the most appropriate category. If none apply, classify it as 'Uncategory'.\n"
        "Output strictly in the following format:\n"
        "Class: [Crack/Efflorescence/Scaling/Corrosion/Uncategory]\n"
        "Description: [classification evidence]"
    )

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are an expert in analyzing building defects."},
                {"role": "user", "content": prompt},
            ],
            max_tokens=150
        )

        response_text = response.choices[0].message.content.strip()
        print(f"Classification response (description: {description}): {response_text}")

        match = re.search(r"Class:\s*(\w+)\s*Description:\s*(.+)", response_text, re.DOTALL)
        if match:
            category = match.group(1).strip()
            detailed_description = match.group(2).strip()
            return category, detailed_description
        else:
            return "Uncertain", "Invalid classification response format"

    except Exception as e:
        print(f"Classification request failed: {e}")
        return "Uncertain", "Request failed"


def categorize_images_by_description(image_descriptions):
    """Categorize images based on their descriptions"""
    categorized_images = {}
    descriptions_with_categories = {}

    for image_name, description in image_descriptions.items():
        category, detailed_description = classify_and_describe_image(description)
        categorized_images[image_name] = category
        descriptions_with_categories[image_name] = {
            "category": category,
            "description": detailed_description
        }

    return categorized_images, descriptions_with_categories


# ==================== Image relocation ====================
def move_images_by_category(folder_path, categorized_images):
    """Move images into category-specific folders"""
    categories = ['Crack', 'Efflorescence', 'Scaling', 'Corrosion', 'Uncertain']
    category_folders = {cat: os.path.join(folder_path, cat) for cat in categories}

    for folder in category_folders.values():
        if not os.path.exists(folder):
            os.makedirs(folder)

    for image_name, category in categorized_images.items():
        target_folder = category_folders.get(category, os.path.join(folder_path, 'Uncertain'))
        source_path = os.path.join(folder_path, image_name)
        target_path = os.path.join(target_folder, image_name)
        try:
            shutil.move(source_path, target_path)
        except Exception as e:
            print(f"Error moving image {image_name}: {e}")


# ==================== Save CSV ====================
def save_descriptions_to_csv(image_descriptions, descriptions_with_categories, output_file):
    """Save image descriptions and classification results to a CSV file"""
    header = ['ImageName', 'ImgDescription', 'ClassifyReason', 'Category']
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        for image_name, description in image_descriptions.items():
            category_info = descriptions_with_categories.get(image_name, {})
            category = category_info.get('category', 'Uncertain')
            category_description = category_info.get('description', 'Failed to obtain description')
            writer.writerow([image_name, description, category_description, category])


# ==================== Main entry point ====================
if __name__ == "__main__":
    folder_path = r"C:\Users\Ziv\Python_Projects\Project8_Autodefect\autodefect\auto_annotation\dataset\Non_balanced_dataset\FPB"  # Replace with your image folder path
    output_file = r"C:\Users\Ziv\Python_Projects\Project8_Autodefect\autodefect\auto_annotation\dataset\Results\FPB.csv"  # Output CSV path

    # Stage 1: Image encoding
    encoded_images, image_files = encode_images_in_folder(folder_path)

    # Stage 2: Generate descriptions (FBP prompt enhancement)
    image_descriptions = describe_images_in_batches(encoded_images, image_files, batch_size=5)

    # Stage 3: Classification
    categorized_images, descriptions_with_categories = categorize_images_by_description(image_descriptions)

    # Stage 4: Save CSV
    save_descriptions_to_csv(image_descriptions, descriptions_with_categories, output_file)

    # Stage 5: Move images by category
    move_images_by_category(folder_path, categorized_images)
